In [144]:
# trnsl.1.1.20191109T120949Z.0b64f5bbabf5efcd.806496e582cf38c63bef5b7590f3fdf6ea939e2a

In [145]:
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass

import requests
import pandas as pd
import numpy as nm

### to see process info

In [146]:
class nice_out:
    def __init__(self, L):
        self.L = L
        self.counter = 0
        self.per = 0
    def update(self):
        if(self.per == (self.counter*100)//self.L):
            print("\r" + str(self.per) + "%" + " " + "#"*(self.per//2) + "_"*(50 - self.per//2), end = "")
            self.per += 1
        self.counter += 1
    def clear(self):
        print("\r" + " "*100, end = "\r")

## Functions

In [147]:
# request.text = {"code":200,"lang":"en-ru","text":["OUTPUT"]}
def parse_request(request):
    text = request.text
    text = text[text.find("text") + 8: len(text) - 3].replace('\\', '')
    return text

def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
def list2set(x):
    return set(x)

# Yandex translator

In [151]:
#key - your key for Yandex translator
def translate_data(key, path_to_source, path_to_result="Rus_data_Yandex.csv"):
    URL = 'https://translate.yandex.net/api/v1.5/tr.json/translate?'
    translator = {'key': key, 'text': "", 'lang': "en-ru"}
    df = pd.read_csv(path_to_source)[['Event','Xintent','Xemotion','Otheremotion']]
    cnt = nice_out(df.shape[0])
    for i in range(0, df.shape[0]):
        cnt.update()
        to_translate = df.iloc[i]["Event"] + "&#&" + df.iloc[i]["Xintent"] + "&#&" + df.iloc[i]["Xemotion"] + "&#&" + df.iloc[i]["Otheremotion"]
        translator["text"] = to_translate
        translated = parse_request(requests.post(URL, translator))
        eve, xin, xre, oth = translated.split("&#&")
        
        df.iloc[i]["Event"]        = eve
        df.iloc[i]["Xintent"]      = xin
        df.iloc[i]["Xemotion"]     = xre
        df.iloc[i]["Otheremotion"] = oth
    cnt.clear()
    df.to_csv(path_to_result, encoding="utf-8")
    
    print(df.head())

In [152]:
URL = 'https://translate.yandex.net/api/v1.5/tr.json/translate?'
key = 'trnsl.1.1.20191109T120949Z.0b64f5bbabf5efcd.806496e582cf38c63bef5b7590f3fdf6ea939e2a'
lang = 'en-ru'

text1 = "The dog has a bone"
text2 = "This text has no misprints"
text3 = "I love C++"
text4 = '["excited", "eager", "interested"]'
print(parse_request(requests.post(URL, data={'key': key, 'text': text1, 'lang': lang})))
print(parse_request(requests.post(URL, data={'key': key, 'text': text2, 'lang': lang})))
print(parse_request(requests.post(URL, data={'key': key, 'text': text3, 'lang': lang})))
print(parse_request(requests.post(URL, data={'key': key, 'text': text4, 'lang': lang})))

У собаки есть кость
В этом тексте нет опечатки
Я люблю с++
["возбуждение", "хочется", "интересно"]


## Russian Dataset

#### params:
#### path_to_source - way (and the name) to english file on your computer
#### path_to_result - way (and the name) to file where russian database will be saved

In [153]:
translate_data(key, '/home/fomius2000/Documents/Python_proj/NLP/event2mind/dev.csv')

                        Event  Xintent Xemotion             Otheremotion                            
0  Она усаживается при стирке  ["нет"]  ["нет"]  ["расстроен", "злится"]
1  Она усаживается при стирке  ["нет"]  ["нет"]  ["расстроен", "злится"]
2  Она усаживается при стирке  ["нет"]  ["нет"]           ["отягчающих"]
3         Он наконец-то нашел  ["нет"]  ["нет"]            ["спокойную"]
4         Он наконец-то нашел  ["нет"]  ["нет"]          ["возбуждение"]
